In [46]:
import pandas as pd
import numpy as np

In [51]:
data = pd.read_csv('anime.csv')
print(data.shape)

(1440, 10)


In [52]:
#data

In [53]:
data.head()

,Page,Rus_name,Eng_name,Href,Genres,Rating,Description,Imgs,Studios,Minor_names
0,1,Стальной алхимик: Братство смотреть онлайн,Hagane no Renkinjutsushi (2009) / FMA (2009) с...,/catalog/hagane-no-renkinjutsushi-2009-2400,"['Экшен', 'Военное', 'Приключения', 'Комедия',...",9.2,['Описание анимеРемейк одноимённого аниме-сери...,/posters/2400.10127424713.jpg,['/anime/studios/4-bones'],Стальной алхимик: Братство / Hagane no Renkinj...
1,1,Атака титанов: Финал смотреть онлайн,Shingeki no Kyojin: The Final Season смотреть ...,/catalog/anime-21166,[],NaN,[],NaN,[],NaN
2,1,Врата Штейна смотреть онлайн,Steins;Gate смотреть онлайн,/catalog/steins-gate-914,"['Фантастика', 'Триллер']",9.1,['Описание анимеДействие сериала происходит ле...,/posters/914.11477207553.jpg,['/anime/studios/314-white-fox'],Врата Штейна / シュタインズ・ゲート / Steins Gate / シュタイ...
3,1,Атака титанов 3. Часть 2 сезон смотреть онлайн,Shingeki no Kyojin Season 3 Part 2 / SnK3.5 см...,/catalog/anime-19608,[],NaN,[],NaN,[],NaN
4,1,Гинтама 4 сезон смотреть онлайн,Gintama° смотреть онлайн,/catalog/gintama-9815,"['Экшен', 'Комедия', 'Исторический', 'Пародия'...",9.1,['Описание анимеНикто этого уже не ждал и не н...,/posters/9815.35700322133.jpg,['/anime/studios/1258-bandai-namco-pictures'],Gintama° / Гинтама 4 / Gintama (2015) / 銀魂゜ / ...


### Необходимые изменения данных

 - Восстановить рейтинг аниме по его соседям и переписать все записи с нерабочими ссылками в отдельную таблицу      
 - Удалить "смотреть онлайн" после названия и "Описание аниме" перед описанием
 - Постараться выделить в отдельный столбец информацию о годе выпуска аниме по одному из названий 
 - Постараться добавить столбец с номером сезона(нужно больше информации о формате)
 - Из нескольких английских названий оставить только одно, остальные добавить к альтернативным
 - Т.к. описаний может быть максимум 2: добавить доп.столбец для описания и разделить информацию из описаний на 2 столбца
 - Из ссылки на студию выделить только название(после номера и дефиса)
